# Functions testing
Just try to wrong all the functions available in the py files

In [1]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path

import numpy as np
import pandas as pd
import wrds

from settings import config

DATA_DIR = Path(config("DATA_DIR"))
WRDS_USERNAME = config("WRDS_USERNAME")
START_YEAR = config("START_YEAR")
END_YEAR = config("END_YEAR")

In [7]:
int(START_YEAR)

2001

In [22]:
def pull_markit_data(start_year=START_YEAR, end_year=END_YEAR, wrds_username=WRDS_USERNAME):
    db = wrds.Connection(wrds_username = wrds_username)
    df = pd.DataFrame()

    for year in range(int(start_year), int(end_year)):
        table = f"markit.cds{year}"
        query=f"""
        WITH first_trade AS (
    SELECT 
        ticker, 
        redcode, 
        DATE_TRUNC('month', date) AS month,  -- Extract month
        date AS first_trade_date,            -- Store actual trade date
        parspread,                           -- CDS spread
        ROW_NUMBER() OVER (
            PARTITION BY ticker, redcode, DATE_TRUNC('month', date) 
            ORDER BY date ASC
        ) AS rn  -- Rank by earliest trade date
    FROM 
        {table}
    WHERE 
        tenor = '5Y' 
        AND country = 'United States'
)
SELECT 
    ticker, 
    redcode, 
    month, 
    first_trade_date, 
    parspread AS midspread
FROM 
    first_trade
WHERE 
    rn = 1  -- Select only the first trade for each firm per month
        """
        new_df = db.raw_sql(query)
        df = pd.concat([df, new_df])

    db.close()
    return df


In [23]:
df = pull_markit_data()

Loading library list...
Done


In [24]:
df.to_parquet(DATA_DIR / "Markit_CDS.parquet")

In [20]:
import pandas as pd
import requests
from io import BytesIO
from pathlib import Path
import settings

from settings import config
import io

In [19]:
def pull_fed_yield_curve():
    """
    Download the latest yield curve from the Federal Reserve
    
    This is the published data using Gurkaynak, Sack, and Wright (2007) model
    """
    
    url = "https://www.federalreserve.gov/data/yield-curve-tables/feds200628.csv"
    response = requests.get(url)
    pdf_stream = BytesIO(response.content)
    df = pd.read_csv(pdf_stream, skiprows=9, index_col=0, parse_dates=True)
    cols = ['SVENY' + str(i).zfill(2) for i in range(1, 5)]
    return df[cols]

def pull_swap_rates():
    urls = {
        "DGS6MO": "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=803&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DGS6MO&scale=left&cosd=1981-09-01&coed=2025-03-12&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-03-14&revision_date=2025-03-14&nd=1981-09-01",
        "DGS3MO": "https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=803&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=DGS3MO&scale=left&cosd=1981-09-01&coed=2025-03-12&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Daily&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-03-14&revision_date=2025-03-14&nd=1981-09-01"
    }

    dataframes = {}
    for key, url in urls.items():
        response = requests.get(url)
        response.raise_for_status()  
        
        df = pd.read_csv(io.StringIO(response.text), parse_dates=["observation_date"])
        df.columns = ["observation_date", key]  
        dataframes[key] = df

    # Merge dataframes on DATE
    df_merged = dataframes["DGS3MO"].merge(dataframes["DGS6MO"], on="observation_date", how="outer")
    df_merged = df_merged.rename(columns = {"observation_date": "Date"})

    return df_merged

In [21]:
df = pull_fed_yield_curve()
path = Path(DATA_DIR) / "fed_yield_curve.parquet"
df.to_parquet(path)

df_fred = pull_swap_rates()
path_fred = Path(DATA_DIR) / "swap_rates.parquet"
df_fred.to_parquet(path_fred)
